In [1]:
import os
os.environ["KERAS_BACKEND"] = "jax"

In [2]:
import tensorflow as tf
import numpy as np
import json
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def data_generator(filepath, tokenizer, batch_size=32, max_length=100, test_size=0.2):
    """
    A generator function that yields batches of tokenized and padded sequences and their labels.
    
    Parameters:
    - filepath: Path to the JSONL file.
    - tokenizer: An instance of tf.keras.preprocessing.text.Tokenizer.
    - batch_size: The number of samples to return in each batch.
    - max_length: The maximum length of the sequences after padding.
    - test_size: The proportion of the dataset to include in the test split.
    
    Yields:
    - A tuple (batch_sequences, batch_labels), where:
        - batch_sequences is a numpy array of tokenized and padded sequences.
        - batch_labels is a numpy array of labels for each sequence in the batch.
    """
    titles = []
    view_counts = []

    with open(filepath, 'r', encoding='utf-8') as file:
        for line in tqdm(file, desc="Loading and processing data"):
            record = json.loads(line)
            titles.append(record['title'])
            view_counts.append(record['view_count'])
            
            if len(titles) == batch_size:
                sequences = tokenizer.texts_to_sequences(titles)
                padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
                labels = np.log(np.array(view_counts, dtype=np.float32))
                labels = np.where(labels == -np.inf, 0, labels)
                
                yield padded_sequences, labels
                
                titles = []
                view_counts = []
                
    if titles:
        sequences = tokenizer.texts_to_sequences(titles)
        padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
        labels = np.log(np.array(view_counts, dtype=np.float32))
        labels = np.where(labels == -np.inf, 0, labels)
        
        yield padded_sequences, labels

def sample_titles(filepath, sample_size=1000):
    """
    Reads a sample of titles from a JSONL file.

    Parameters:
    - filepath: Path to the JSONL file.
    - sample_size: Number of titles to sample.
    
    Returns:
    - A list of sampled titles.
    """
    titles = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            if len(titles) >= sample_size:
                break
            record = json.loads(line)
            titles.append(record['title'])
    return titles


max_length = 100
filepath = '/mnt/datassd/processed_file.jsonl'

titles_sample = sample_titles(filepath)


tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, oov_token="<OOV>")

tokenizer.fit_on_texts(titles_sample)

#tokenizer.fit_on_texts(titles)

2024-03-12 17:45:19.688342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 17:45:19.688389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 17:45:19.689432: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-12 17:45:19.695171: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 17:45:20.270355: W tensorflow/compiler/tf2

In [3]:
import mlflow
import keras
import keras_nlp
from keras import layers

vocab_size = 10000  # Adjust based on your vocabulary size
embedding_dim = 256
max_length = 100  # Adjust based on your titles' maximum length
num_heads = 8  # Number of attention heads in the Transformer encoder
intermediate_dim = 512  # Dimensionality of the encoder's intermediate (feed-forward) layer

# Define input layer
inputs = keras.Input(shape=(max_length,), dtype='int64')

# Token and position embedding layer
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=vocab_size,
    sequence_length=max_length,
    embedding_dim=embedding_dim,
)
x = embedding_layer(inputs)

# Transformer encoder layer
encoder = keras_nlp.layers.TransformerEncoder(
    num_heads=num_heads,
    intermediate_dim=intermediate_dim,
    activation='relu',
    dropout=0.1,
)
x = encoder(x)

# GlobalMaxPooling1D layer for regression task
x = layers.GlobalMaxPooling1D()(x)

# Additional dense layers
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(1, activation='linear')(x)  # Linear activation for regression

# Compile the model
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss='mean_squared_error')
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, 100, 256)       │     2,585,600 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, 100, 256)       │       527,104 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,178,753 (12.13 MB)

 Trainable params: 3,178,753 (12.13 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:

# MLflow tracking
mlflow.autolog()

epochs = 10
batch_size = 1024  # This is now only for your reference and generator configuration

with mlflow.start_run():
    # Assuming your data_generator now correctly configures batches of size `batch_size`
    model.fit(x=data_generator(filepath, tokenizer, batch_size, max_length),
              epochs=epochs,
              steps_per_epoch=83419,verbose = 0)  # Make sure this matches your actual number of batches per epoch)

    # Log additional metrics or parameters
    mlflow.log_param("vocab_size", vocab_size)
    mlflow.log_param("embedding_dim", embedding_dim)
    mlflow.log_param("max_length", max_length)
    mlflow.log_param("num_heads", num_heads)
    mlflow.log_param("intermediate_dim", intermediate_dim)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("generator_batch_size", batch_size)  # Renamed to clarify this is the generator's batch size
    
    # Save and log the model in MLflow
    model_name = "YT_Transformer"
    model_path = f"{model_name}.keras"
    model.save(model_path)
    mlflow.keras.log_model(model, "model", registered_model_name=model_name)



2024/03/12 17:45:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/03/12 17:45:36 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/03/12 17:45:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
Loading and processing data: 8it [00:00, 55.45it/s]

Epoch 1/15


Loading and processing data: 14it [00:00, 53.26it/s]2024/03/12 17:45:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/jay/.local/lib/python3.11/site-packages/keras/src/backend/jax/core.py:73: UserWarning: Explicitly requested dtype int64 requested in asarray is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more."


   1/1000 ━━━━━━━━━━━━━━━━━━━━ 1:50:41 7s/step - loss: 88.3736

Loading and processing data: 20it [00:06,  2.07it/s]

...

Loading and processing data: 59838it [00:30, 2306.84it/s]

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 5.0862


2024/03/12 17:46:07 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: cannot import name '_initialize_variables' from 'keras.src.backend' (/home/jay/.local/lib/python3.11/site-packages/keras/src/backend/__init__.py)


AttributeError: module 'mlflow' has no attribute 'keras'

Loading and processing data: 59838it [00:50, 2306.84it/s]

In [ ]:
#model.save(f'{actual_model_name}.keras')

predictions = model.predict(X_test)

for i in range(10):  # Display first 10 predictions
    print(f"Predicted view count: {predictions[i]}, Actual view count: {y_test[i]}")

In [ ]:
# Make a line 
x = np.linspace(0,10,100)
y = np.linspace(0,10,100)

In [ ]:
#import seaborn as sns
#import matplotlib.pyplot as plt
#
#
#heatmap, xedges, yedges = np.histogram2d(y_test.flatten(), predictions.flatten(), bins=100)
#
#extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
#
#
#plt.imshow(heatmap.T, extent=extent, origin='lower')
#plt.plot(x,y, 'r--')
#plt.xlabel('Views Order of Magnitude')
#plt.ylabel('Predicted Order of Magnitude')
#plt.xlim(2,15)
#plt.ylim(0,17)
#plt.savefig(f'{actual_model_name}_heatmap_bonito.png')
#
#import matplotlib.pyplot as plt
#
#
#plt.scatter(y_test, predictions, alpha=0.1, s=0.5)
#plt.plot(x,y,'r--')
#plt.xlabel('Actual View Count')
#plt.ylabel('Predicted View Count')
#plt.savefig(f'{actual_model_name}_scatter_bonit.png')

In [ ]:
# If you need to convert an array of values
y_test_e = np.exp(y_test)  # Assuming y_test was in loge form
y_test_10 = np.log10(y_test_e)

predictions_e = np.exp(predictions)  # Assuming predictions were in loge form
predictions_10 = np.log10(predictions_e)

y_test = y_test_10
predictions = predictions_10

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_test and predictions are available and in log form
# Heatmap
heatmap, xedges, yedges = np.histogram2d(y_test.flatten(), predictions.flatten(), bins=100)
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.figure(figsize=(10, 8))
sns.set(style="white")

# Using a colormap (e.g., 'viridis' which is visually appealing and colorblind-friendly)
plt.imshow(heatmap.T, extent=extent, origin='lower', aspect='auto', cmap='viridis')

# Assuming x, y for the red dashed line are defined correctly and correspond to log scale
plt.plot(x, y, 'r--')

plt.xlabel('Log of Actual View Count')
plt.ylabel('Log of Predicted View Count')
plt.colorbar(label='Count of Test')
plt.title('Heatmap of Predictions vs Actual Views')
plt.xlim(0, 9)
plt.ylim(0, 9)

# Adjusting x and y axis to show in 10^ format
ax = plt.gca()
ax.set_xticklabels([f'$10^{{{int(float(label))}}}$' for label in ax.get_xticks()])
ax.set_yticklabels([f'$10^{{{int(float(label))}}}$' for label in ax.get_yticks()])

plt.savefig(f'{actual_model_name}_heatmap_bonito.png')

In [ ]:
plt.figure(figsize=(10, 8))
sns.set(style="whitegrid")

# Scatter plot with adjustments for alpha and size for better visibility
plt.scatter(y_test, predictions, alpha=0.2, s=10, cmap='viridis')

plt.plot(x, y, 'r--')  # Assuming x, y for the red dashed line are correct

plt.xlabel('Log of Actual View Count')
plt.ylabel('Log of Predicted View Count')
plt.title('Scatter Plot of Predicted vs Actual Views')
plt.xlim(0, 9)
plt.ylim(0, 9)


# Adjust axis to reflect 10^x and 10^y
ax = plt.gca()
ax.set_xticklabels([f'$10^{{{int(float(label))}}}$' for label in ax.get_xticks()])
ax.set_yticklabels([f'$10^{{{int(float(label))}}}$' for label in ax.get_yticks()])

plt.savefig(f'{actual_model_name}_scatter_bonito.png')